# **Guia Interativo: Simulação de Rede Quântica para Criação de Estados GHZ**

Este notebook é um guia para entender e interagir com o projeto de simulação de rede quântica. O objetivo principal desta simulação é criar um estado GHZ (Greenberger-Horne-Zeilinger) entre um nó central (Hub) e um conjunto de nós sensores.

Aqui, vamos explorar:
1.  O **fluxo da simulação principal**, que executa o protocolo para todos os nós configurados.
2.  Um **exemplo prático interativo**, onde simulamos o protocolo para um par aleatório de sensores.

## 2. O Fluxo da Simulação Principal (Na Prática)

Agora, vamos executar as etapas essenciais que o script `main.py` realiza, mas de forma controlada e explicada aqui no notebook. Isso nos permitirá inspecionar os componentes da simulação antes de executá-la.

In [1]:
# Célula 1: Importações

# Ferramentas da simulação
from sequence.topology.router_net_topo import RouterNetTopo

# Nossas aplicações e o arquivo de configuração
from qsn.app.ghz_active import HubGHZActiveApp, SensorGHZActiveApp
from qsn.parameters import set_parameters, CONFIG

# Ferramentas de Log (opcional para o guia, mas bom ter)
from qsn.utils import setup_logger

print("Módulos importados com sucesso!")

Módulos importados com sucesso!


### 2.1. Carregando a Topologia

O primeiro passo é carregar a estrutura da nossa rede. Usamos o caminho do arquivo definido em `CONFIG` e o passamos para o `RouterNetTopo`, que montará todos os nós, canais quânticos e outros componentes de hardware para nós.

In [2]:
# Célula 2: Carregando a topologia

network_file = CONFIG['simulacao']['NETWORK_CONFIG_FILE']

print(f"Carregando a rede a partir de '{network_file}'...")
network_topo = RouterNetTopo(network_file)

# A timeline é o "relógio" da nossa simulação
tl = network_topo.get_timeline()

print("Topologia da rede carregada.")

Carregando a rede a partir de 'qsn/net.json'...
Topologia da rede carregada.


### 2.2. Configurando o Sistema de Logs

Antes de executar qualquer coisa, precisamos configurar nosso sistema de gravação de eventos (logs). Para isso, usamos a função `setup_logger`, que funciona como um "diretor de câmeras", decidindo o que será gravado.

Ela possui dois modos:
* **`verbose`**: Grava absolutamente tudo que acontece na simulação, incluindo os detalhes internos da biblioteca SeQUeNCe. Ótimo para depuração profunda.
* **`custom`**: Grava apenas os eventos das nossas aplicações (`hub_app`, `sensor_app`, etc.), gerando um log mais limpo e focado na nossa lógica.

Vamos usar o modo `custom` para manter nosso log focado.

In [3]:
print("Configurando o logger para a simulação...")
setup_logger(tl, CONFIG['simulacao']['LOG_FILE_NAME'], mode='custom')

Configurando o logger para a simulação...


### 2.3. Configurando os Parâmetros do Hardware

Com a topologia em memória, o próximo passo é configurar as propriedades físicas de cada componente, como a fidelidade das memórias quânticas, a eficiência dos detectores, etc. A função `set_parameters` faz isso, lendo os valores do nosso dicionário `CONFIG` e aplicando-os a cada componente da rede.

In [4]:
# Célula 3: Aplicando os parâmetros

print("Aplicando parâmetros de hardware (memórias, detectores, etc.)...")
set_parameters(network_topo)
print("Parâmetros aplicados.")

Aplicando parâmetros de hardware (memórias, detectores, etc.)...
Parâmetros aplicados.


### 2.4. Visualizando a Rede Configurada

Para ter certeza de que entendemos a estrutura da nossa simulação, podemos gerar um "mapa" simples baseado na nossa configuração. O código abaixo lê a configuração dos hubs e mostra quais sensores estão logicamente conectados a cada um.

In [5]:
# Célula 4: Visualizando a estrutura lógica

print("Estrutura Lógica da Rede:")
for hub_info in CONFIG['hubs_config']:
    hub_name = hub_info['name']
    print(f"└── Hub: {hub_name}")
    
    sensor_names = hub_info['sensors']
    for i, sensor_name in enumerate(sensor_names):
        # Verifica se é o último sensor para desenhar o conector correto
        if i == len(sensor_names) - 1:
            print(f"    └── Sensor: {sensor_name}")
        else:
            print(f"    ├── Sensor: {sensor_name}")

Estrutura Lógica da Rede:
└── Hub: Hub1
    ├── Sensor: Sensor1H1
    ├── Sensor: Sensor2H1
    ├── Sensor: Sensor3H1
    └── Sensor: Sensor4H1
└── Hub: Hub2
    ├── Sensor: Sensor1H2
    ├── Sensor: Sensor2H2
    ├── Sensor: Sensor3H2
    └── Sensor: Sensor4H2
└── Hub: Hub3
    ├── Sensor: Sensor1H3
    ├── Sensor: Sensor2H3
    ├── Sensor: Sensor3H3
    └── Sensor: Sensor4H3


## 3. Exemplo Prático: Simulação com um Par Aleatório

Nesta seção, vamos realizar uma simulação focada. Em vez de ativar todos os hubs e sensores, faremos o seguinte:
1.  Escolheremos um hub específico para ser o nosso orquestrador.
2.  Selecionaremos aleatoriamente 2 sensores que estão conectados a esse hub.
3.  Instalaremos as aplicações *apenas* nesses 3 nós (1 hub e 2 sensores).
4.  Executaremos a simulação e veremos o que acontece.

Isso nos permite testar o protocolo em uma escala menor e de forma dinâmica.

### 3.1. Selecionando os Nós Aleatoriamente

Primeiro, definimos qual hub queremos usar. Depois, olhamos para a nossa `CONFIG` para ver a lista de sensores daquele hub e usamos a biblioteca `random` para sortear dois deles.

In [6]:
# Célula 5: Seleção aleatória dos participantes
import random

# Você pode mudar este nome para "Hub2" ou "Hub3" para testar com outros!
target_hub_name = "Hub1"
hub_info = None

# Encontra a configuração do nosso hub alvo no dicionário CONFIG
for h_info in CONFIG['hubs_config']:
    if h_info['name'] == target_hub_name:
        hub_info = h_info
        break

if hub_info:
    # Pega a lista de sensores do hub
    possible_sensors = hub_info['sensors']
    
    # Sorteia 2 sensores da lista, sem repetição
    if len(possible_sensors) >= 2:
        selected_sensors = random.sample(possible_sensors, 2)
        
        print(f"Hub selecionado: {target_hub_name}")
        print(f"Sensores sorteados para o experimento: {selected_sensors[0]} e {selected_sensors[1]}")
    else:
        print(f"Erro: O {target_hub_name} não tem pelo menos 2 sensores configurados.")
else:
    print(f"Erro: Hub com o nome '{target_hub_name}' não encontrado na configuração.")

Hub selecionado: Hub1
Sensores sorteados para o experimento: Sensor3H1 e Sensor4H1


### 3.2. Instalando as Aplicações

Agora, vamos instalar a `HubGHZActiveApp` no nosso hub alvo e a `SensorGHZActiveApp` em cada um dos dois sensores sorteados. Note que a aplicação do hub é informada que ela só precisa se preocupar com esses dois sensores.

In [7]:
# Célula 6: Instalando as aplicações no cenário específico

# Pega todos os nós do tipo QUANTUM_ROUTER, que já vem como uma lista de objetos.
all_nodes = network_topo.get_nodes_by_type(RouterNetTopo.QUANTUM_ROUTER)
node_map = {node.name: node for node in all_nodes}

# Pega os objetos dos nós que selecionamos (esta parte já estava correta)
hub_node = node_map.get(target_hub_name)
sensor_node1 = node_map.get(selected_sensors[0])
sensor_node2 = node_map.get(selected_sensors[1])

# Verifica se todos os nós foram encontrados antes de prosseguir
if hub_node and sensor_node1 and sensor_node2:
    # Instala a App no Hub
    hub_app = HubGHZActiveApp(hub_node, selected_sensors, 
                              CONFIG['simulacao']['START_TIME'], 
                              CONFIG['simulacao']['END_TIME'])
    hub_node.set_app(hub_app)
    print(f"Aplicação instalada no Hub: {hub_node.name}")

    # Instala as Apps nos Sensores
    sensor_app1 = SensorGHZActiveApp(sensor_node1)
    sensor_node1.set_app(sensor_app1)
    print(f"Aplicação instalada no Sensor: {sensor_node1.name}")

    sensor_app2 = SensorGHZActiveApp(sensor_node2)
    sensor_node2.set_app(sensor_app2)
    print(f"Aplicação instalada no Sensor: {sensor_node2.name}")
else:
    print("Erro: Um ou mais nós selecionados não foram encontrados na topologia.")

Aplicação instalada no Hub: Hub1
Aplicação instalada no Sensor: Sensor3H1
Aplicação instalada no Sensor: Sensor4H1


### 3.3. Executando a Simulação

Com as aplicações instaladas, podemos iniciar o "relógio" da simulação (`tl.init()`), dar o comando para o Hub começar o protocolo (`hub_app.start()`) e rodar a simulação até o fim (`tl.run()`).

In [8]:
# Célula 7: Executando a simulação focada

print(f"\nIniciando simulação focada entre {target_hub_name} e os sensores {selected_sensors}...")

# Reinicia a timeline para um estado limpo
tl.init()

# Apenas o nosso hub selecionado inicia o processo
hub_app.start()

# Roda a simulação
tl.run()

print("\nSimulação focada concluída!")
print(f"Verifique o arquivo '{CONFIG['simulacao']['LOG_FILE_NAME']}.txt' para ver os detalhes da comunicação.")


Iniciando simulação focada entre Hub1 e os sensores ['Sensor3H1', 'Sensor4H1']...

Simulação focada concluída!
Verifique o arquivo 'log.txt' para ver os detalhes da comunicação.
